###### Smali & Manifest File Analysis
###### This notebook will try to analyze a directory and its SMALI and MANIFEST files. The analysis will find and print sensitive methods and packages used by the program.

###### This particular notebook only does 1 directory at a time.

In [1]:
from os import walk
import pandas as pd
import os

In [2]:
def decompileAll():

    # 1: Traverse all APKs in the folder:
    dir = 'C:\\Users\\Vin\\Desktop\\python-ransomware-analyzer\\ransomware'

    for filename in os.listdir(dir):
        if filename.endswith('.apk'):
            print("decompiling: "+ filename)
            
            #set output directory (removing .apk from the end)
            foldername = 'out_' + filename[:-4]
            var1 = r'C:\Users\Vin\Desktop\python-ransomware-analyzer\decompiledApks\\' + foldername
        
            #set input apk
            var2 = r'C:\Users\Vin\Desktop\python-ransomware-analyzer\ransomware\\' + filename

            #convert python vars to batch vars
            os.putenv("VAR1", var1)
            os.putenv("VAR2", var2)

            #call the run.bat for the apktool
            os.system("run.bat")
    print("...Done!")

In [3]:
#create empty lists
methods_to_find = []
perms_to_find = []

#read file using pandas
df = pd.read_csv('functions.csv', engine='python')
methods_to_find = df.functions

df = pd.read_csv('perms.csv', engine='python')
perms_to_find   = df.perms

In [4]:
#Create class object to be used in the lists
class obj:
    def __init__(self, name):
        self.name = name
        self.count = 0
    def print(self):
        print(self.name + ": " + str(self.count))

In [5]:
#Initialize empty lists
myList = []
fakeList = []

#Fill lists with permissions and functions
for i in perms_to_find:
    myList.append(obj(i))
    fakeList.append(i)

for i in methods_to_find:
    myList.append(obj(i))
    fakeList.append(i)

#self print function for list
def printList(list):
    for i in range(len(list)):
        try:
            list[i].print()
        except AttributeError:
            print(list[i])

Everything up to here works dandy!

In [6]:
#Create empty dataframe
df = pd.DataFrame(columns=fakeList)

#this clears the CSV if it existed already
df.to_csv('my_csv.csv', mode='w', header=True)

In [7]:
def findFunctionsInDir(direct, list=0, verbose = True):

    #for each directory, and files
    for dirpath, dirnames, files in walk(direct):
        
        #skip the res folder, too many subfolders, slows.
        if('res' in dirpath): continue

        if(verbose == True): print(f'Found directory: {dirpath}')
        for file_name in files:
            
            #if it is a manifest file
            if('.xml' in file_name and 'anifest' in file_name):
                
                #print file name
                if(verbose == True): print(file_name)
                #print("XML found")

                #open file and print line number and contents
                #I had to use latin encoding here or it would simply not work.
                with open(dirpath + "\\" + file_name, encoding="latin-1") as myFile:
                    for num, line in enumerate(myFile, 1):
                        for i in perms_to_find:
                            if(i in line and ('<uses-permission android:name=' in line)
                                         or  ('<action android:name="android.app.action') in line):
                                
                                #Set permission found to 1
                                #print("found permission!")
                                myList[fakeList.index(i)].count = 1

            #if it is a smali file
            if('.smali' in file_name):
                
                #print file name
                if(verbose == True): print(file_name)

                #open file and print line number and contents
                with open(dirpath + "\\" + file_name) as myFile:
                    for num, line in enumerate(myFile, 1):
                        for i in methods_to_find:
                            if(i in line and 'invoke' in line):

                                #Increase counter for found item
                                myList[fakeList.index(i)].count += 1
                                #myList[fakeList.index(i)].print()

In [8]:
#function saves to csv file
def saveToFile(df):
    df.to_csv('my_csv.csv', mode='a', header=False)  

In [9]:
def appendToDF(list, df):
    #initialize empty list for the counts only.
    listOfCount = []

    #fill such list
    for i in range(len(list)):
        listOfCount.append(list[i].count)

    #append the list to the dataframe
    df.loc[len(df.index)] = listOfCount

In [10]:
#for the passed directory, do: CALL FIND FUNCTIONS, APPEND THE RESULTING myList to the df, save.
def runAnalysis(filename):
    findFunctionsInDir(direct=filename, verbose=False)
    
    #see if list checks out:
    #printList(myList)
    
    appendToDF(myList, df)

    #TODO instead of this, we can save a pre-made list and substitute it.
    myList.clear()
    for k in perms_to_find:
        myList.append(obj(k))

    for k in methods_to_find:
        myList.append(obj(k))

In [11]:
#decompile all of the APKs in folder (hardcoded)
#decompileAll()

#This line will clear the DF (call if needed)
df = df.iloc[0:0]

direc = "C:\\Users\\Vin\\Desktop\\python-ransomware-analyzer\\decompiledApks"

# Here, filename means folder.
for filename in os.listdir(direc):
    print("Analyzing: " + filename)

    try:
        #IF THINGS DONT WORK, CHECK IF THE PATH IS CORRECT:
        runAnalysis(os.path.join(direc, filename))
    except UnicodeDecodeError:
        print("Unicode Error")
    except FileNotFoundError:
        print("File Not Found!")
    

saveToFile(df)
print("..done! Saved to csv.")

decompiling: 00357b0e208c20df3182d54cb2ba15bf.apk
decompiling: 0035d75d5b143a92d5b55c5063f63bd0.apk
decompiling: 0064a447770d0045673ed2a64e129636.apk
decompiling: 006a476515e90faeb71753a410a8a0d9.apk
decompiling: 008aafc94cfdc5e41d8f5fa41abe23ac.apk
decompiling: 00c0a3916b33e792e4b5f23a3a58ec7d.apk
decompiling: 00da61a6c82ee1b6321898c40dda0205.apk
decompiling: 00f6cb935df075494a1fd1ce5e918a7a.apk
decompiling: 01147ee72ad07cb9c1416b9963e56f99.apk
decompiling: 01441e0741ed81400cad56bfc3500754.apk
decompiling: 0183b868998ea7ca75d5a78136c2e977.apk
decompiling: 0187bcbef201d680b9607c999402ba89.apk
decompiling: 01cde31e6ddf3429cd1d0e0536631f2d.apk
decompiling: 01fa7c4bb9adfca5262a3f48256e59cc.apk
decompiling: 021b91b6f0ef6c51a2c7c7aece08b837.apk
decompiling: 02548535ff1cc285fddf699f2d77bcba.apk
decompiling: 02742daa4a1af89abc304131d09e90c6.apk
decompiling: 029841c91466bd44745ac00b8e5b73c5.apk
decompiling: 02985692e377d55eaa4347a77b52d789.apk
decompiling: 02cce0f8a26ca3f584242e9edac41423.apk


In [12]:
df

,READ_CALENDAR,WRITE_CALENDAR,CAMERA,READ_CONTACTS,WRITE_CONTACTS,GET_ACCOUNTS,ACCESS_FINE_LOCATION,ACCESS_COARSE_LOCATION,RECORD_AUDIO,READ_PHONE_STATE,...,getExternalCacheDir,shareIntent,readPhoneNumber,FileInputStream,getSharedPreferences,telephony,getMessageBody,getDisplayOriginatingAddress,wallpaper,crypto
0,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1785,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,28,5,0,0,0,0
1786,1,1,1,1,1,1,1,1,1,1,...,0,0,0,7,0,0,0,0,0,0
1787,1,1,1,1,1,1,1,1,1,1,...,4,0,0,7,0,5,0,0,0,0
1788,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0


In [13]:
#This line will clear the DF (call if needed)
#df = df.iloc[0:0]

In [14]:
#TODO ADD A PERCENT COMPLETE TO DECOMPILE / ANALYZE